In [34]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, TypeAdapter
import json
import os

In [2]:
# Set the model name for our LLMs.
GEMINI_MODEL = "gemini-1.5-flash"
# Store the API key in a variable.
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
# Initialize the model.
llm = ChatGoogleGenerativeAI(google_api_key=GEMINI_API_KEY, model=GEMINI_MODEL, temperature=0.9)


# List Parser

In [3]:
# Initialize the output parser.
parser = CommaSeparatedListOutputParser()

# Get the output format instructions.
instructions = parser.get_format_instructions()
print(f"instructions: {instructions}")

print()

# Define a query as a string, combining with the instructions.
query = "List 3 cat breeds." + "\n\n" + instructions

# Pass the query to the invoke method, and print the result.
result = llm.invoke(query)
print(result.content)

print()

# Parse the result, store it, and print it.
data = parser.parse(result.content)
print(data)

instructions: Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`

Maine Coon, Siamese, Persian 


['Maine Coon', 'Siamese', 'Persian']


# Structured Parser

In [9]:
# Initialize the model.
llm = ChatGoogleGenerativeAI(google_api_key=GEMINI_API_KEY, model=GEMINI_MODEL, temperature=0.9)

# Define the schemas for our parser.
schemas = [
    ResponseSchema(name="country", description="the country"),
    ResponseSchema(name="capital", description="the capital")
]

# Initialize the output parser using the schema.
parser = StructuredOutputParser.from_response_schemas(schemas)

# Get the output format instructions and print them.
instructions = parser.get_format_instructions(only_json=True)
print(f"Instructions: {instructions}")

print()

# Define a query as a string, combining with the instructions.
query = "Name a country and its capital." + "\n\n" + instructions

# Pass the query to the invoke method, and print the result.
result = llm.invoke(query)
print(f"Iniitial result: {result.content}")

print()

# Parse the result, store it, and print it.
data = parser.parse(result.content)
print(data)
print(data["country"])
print(data["capital"])

print()

# Define a new query using the parsed output.
query = f"What are three tourist attractions in {data['capital']}?"

# Pass the query to the invoke method, and print the result.
result = llm.invoke(query)
print(result.content)

Instructions: 
```json
{
	"country": string  // the country
	"capital": string  // the capital
}
```

Iniitial result: ```json
{
	"country": "France",
	"capital": "Paris"
}
```

{'country': 'France', 'capital': 'Paris'}
France
Paris

Here are three popular tourist attractions in Paris:

1. **The Eiffel Tower:** This iconic wrought-iron lattice tower is a must-see for any visitor to Paris. You can ascend to the top for panoramic views of the city, or simply admire it from afar. 
2. **The Louvre Museum:** Home to some of the world's most famous artworks, including the Mona Lisa, the Louvre is a treasure trove of art and history. 
3. **The Palace of Versailles:** A short train ride from Paris, this opulent palace was once the home of French royalty. Visitors can explore the lavish interiors, stroll through the formal gardens, and learn about the history of the French monarchy. 



# Class-based Parser

In [32]:
# Define the class for parsed responses
class Country(BaseModel):
    name: str = Field(description="the country name")
    capital: str = Field(description="the capital")
    population: int = Field(description="the country's population")

# Initialize the output parser using the Counrty class.
parser = PydanticOutputParser(pydantic_object=Country)

{'properties': {'name': {'description': 'the country name', 'title': 'Name', 'type': 'string'}, 'capital': {'description': 'the capital', 'title': 'Capital', 'type': 'string'}, 'population': {'description': "the country's population", 'title': 'Population', 'type': 'integer'}}, 'required': ['name', 'capital', 'population'], 'title': 'Country', 'type': 'object'}


In [35]:
# Manually create format instructions based on the schema
instructions = """
Please return a JSON object in the following format:
{
    "name": "string",       // the country name
    "capital": "string",    // the capital
    "population": integer   // the country's population
}
"""

print(instructions)
print()

# Define a query as a string, combining with the instructions.
query = "Name any country, its capital, and the country's population." + "\n\n" + instructions

# Invoke the LLM and get the custom response
llm_result = llm.invoke(query)
print(llm_result)

# Parse the result (mocked here) content into a dictionary
try:
    # Parse the LLM response into our Country class
    data = parser.parse(llm_result.content)
    
    # Print the parsed data using Pydantic's model attributes
    print(data)
    print(f"Country Name: {data.name}")
    print(f"Capital: {data.capital}")
    print(f"Population: {data.population}")
    
except Exception as e:
    print(f"Error parsing LLM response: {e}")

print()

# Define a new query using the parsed output.
query = f"What are three tourist attractions in {data.capital}?"

# Normally you would call the LLM again
# result = llm.invoke(query)
# For now, we'll print the query as an example:
print(query)

# Pass the query to the invoke method, and print the result.
result = llm.invoke(query)
print(result.content)


Please return a JSON object in the following format:
{
    "name": "string",       // the country name
    "capital": "string",    // the capital
    "population": integer   // the country's population
}


content='```json\n{\n    "name": "Brazil",\n    "capital": "Brasília",\n    "population": 214766314\n}\n```' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]} id='run-03f5e589-ba93-409e-8539-3b18a3ad1c76-0' usage_metadata={'input_tokens': 69, 'output_tokens': 40, 'total_tokens': 109}
name='Brazil' capital='Brasília' populat

In [14]:
llm_result.content

'```json\n{\n    "name": "France",\n    "capital": "Paris",\n    "population": 67000000\n}\n``` \n'

In [31]:
data = parser.parse(llm_result.content)
data

Country(name='France', capital='Paris', population=67064000)